In [37]:
from datasets import load_dataset

raw_data = load_dataset('yitingxie/rlhf-reward-datasets')
raw_data

Found cached dataset parquet (/root/.cache/huggingface/datasets/yitingxie___parquet/yitingxie--rlhf-reward-datasets-f2627438ff1fb9dd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 219.02it/s]


DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 76256
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 5103
    })
})

In [ ]:
batch = raw_data['train'][:5]
examples_list = []
for i in range(len(batch['prompt'])):
    examples_list.append({
        k:batch[k][i] for k in batch
    })
        
examples_list

In [42]:
import re
def filtering(batch):
    res = {
        'prompt':[],
        'chosen':[],
        'rejected':[]
    }
    examples_list = []
    for i in range(len(batch['prompt'])):
        examples_list.append({
            k:batch[k][i] for k in batch
        })
            
    for example in examples_list:
        q = example['prompt']
        if len(re.findall('Human:',q)) > 1:
            continue
        else:
            q = q.replace(' Human: ','').replace('Human: ','').strip()
            chosen = example['chosen'].replace(' Assistant: ','').replace('Assistant: ','').strip()
            rejected = example['rejected'].replace(' Assistant: ','').replace('Assistant: ','').strip()
        res['prompt'].append(q)
        res['chosen'].append(chosen)
        res['rejected'].append(rejected)
    return res
            
            
filtered_data = raw_data.map(filtering, batched=True)
filtered_data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 23439
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1575
    })
})

In [43]:
filtered_data['test'][:3]

{'prompt': ['Do you know why turkeys became the official food of thanksgiving?',
  'How do I float on my back in the water?',
  "I want to make a pumpkin pie for Thanksgiving, but I've never made one before.  How do I make a pumpkin pie?"],
 'chosen': ['To be honest, I don’t know anything about that. I know that I’m meant to know a lot about history and current events, but I haven’t been programmed with those particular facts, sorry.',
  'You should tuck your arms to your sides and arch your back. Then use your back muscles to raise and lower your body.',
  'There are many variations on pumpkin pie recipes, but here’s one of my favorites:\n\n4 eggs\n\n3⁄4 cup pumpkin puree\n\n1 cup sugar\n\n1⁄2 cup brown sugar\n\n2 cups all-purpose flour\n\n1⁄2 teaspoon baking powder\n\n1⁄2 teaspoon salt\n\n1 teaspoon cinnamon\n\n1 teaspoon ground cloves\n\n1 teaspoon nutmeg\n\n1 teaspoon allspice\n\n1⁄4 teaspoon ground ginger\n\n2/3 cup butter, melted\n\nPreheat oven to 400 degrees. In a medium bowl, 

In [44]:
# 过滤之后test太少了，从train里面挪一点过去
from datasets import concatenate_datasets
filtered_data['test'] = concatenate_datasets([filtered_data['test'],filtered_data['train'].select(range(20000,23439))])
filtered_data['train'] = filtered_data['train'].select(range(0,20000))
filtered_data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 5014
    })
})

In [ ]:
filtered_data.push_to_hub('beyond/rlhf-reward-single-round')

In [ ]:
from datasets import load_dataset
d = load_dataset('beyond/rlhf-reward-single-round-trans_chinese')

In [2]:
d.save_to_disk('../data/rlhf-reward-single-round-trans_chinese')

## BaichuanForSequenceClassification

In [ ]:
from modeling_baichuan_for_cls import BaichuanForSequenceClassification
from peft import PeftModel,get_peft_model
import torch

In [4]:
model = BaichuanForSequenceClassification.from_pretrained(
    'baichuan-inc/baichuan-7B', num_labels=1, 
    torch_dtype=torch.bfloat16, trust_remote_code=True, 
    device_map="auto"
)
# model

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Some weights of the model checkpoint at baichuan-inc/baichuan-7B were not used when initializing BaichuanForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing BaichuanForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BaichuanForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BaichuanForSequenceClassification were not initialized from the model checkpoint at baichuan-inc/baichuan-7B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to us

In [5]:
model = PeftModel.from_pretrained(model, '../weights/baichuan-7B_beyond_rlhf-reward-single-round_-1_peft_last_checkpoint')
# model

对比前后的模型，可以发现，加载LoRA之后，不光是在百川的 W_pack 模块下新增了 LoRA 相关的模块，还在 score 中新增的相关的模块：
```shell
BaichuanForSequenceClassification:
(score): Linear(in_features=4096, out_features=1, bias=False)

PeftModelForSequenceClassification:
(score): ModulesToSaveWrapper(
  (original_module): Linear(in_features=4096, out_features=1, bias=False)
  (modules_to_save): ModuleDict(
    (default): Linear(in_features=4096, out_features=1, bias=False)
  )
)
```
这说明，在训练/保存的时候，LoRA 也训练/保存了最后的 cls head。

当然，这需要在训练模型的时候，指定 LoRA 的任务：
```python

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # <---
    inference_mode=False,
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules = None if script_args.lora_target_models is None else script_args.lora_target_models.split('|')
)

model = BaichuanForSequenceClassification.from_pretrained(
    script_args.model_name, num_labels=1, torch_dtype=torch.bfloat16,trust_remote_code=True, 
    device_map="auto"
)

model = get_peft_model(model, peft_config)
```

In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('baichuan-inc/baichuan-7B', trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [32]:
sent1 = "问：我很好奇，你认为在一场斗争中谁会赢：美洲豹还是土狼？\n\n答：土狼通常会在危险面前逃跑，但它们非常狡猾。美洲豹长着锋利的尖牙，能够捕杀比它们体型大得多的猎物。"
sent2 = "问：我很好奇，你认为在一场斗争中谁会赢：美洲豹还是土狼？\n\n答：你说的“赢”是什么意思？也许你指的是“都更强大”，如果是这样，我认为coyote会赢。虽然jaguar更大，但coyote可以轻松逃跑，并且它更狡猾、奸诈。jaguar可以捉住coyote，但只有当coyote被逼到绝境且筋疲力尽的时候。"
output = model(**tokenizer([sent1,sent2],return_tensors='pt',padding=True))
output.keys()

odict_keys(['logits', 'past_key_values'])

In [33]:
output['logits'], torch.sigmoid(output['logits'])

(tensor([[-1.0391],
         [-0.0732]], dtype=torch.bfloat16, grad_fn=<ToCopyBackward0>),
 tensor([[0.2617],
         [0.4824]], dtype=torch.bfloat16, grad_fn=<SigmoidBackward0>))